In [9]:
import numpy as np
import cv2
from keras.models import load_model
from skimage.transform import resize
from skimage import color, exposure
import warnings
from skimage.feature import hog
warnings.filterwarnings('ignore')


model_path = 'ANN(HOG).h5'
modelq = load_model(model_path)


face_detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

label_dict = {0: 'anger', 1: 'disgust', 2: 'fear', 3: 'happiness', 4: 'neutral', 5: 'sad', 6: 'surprise'}


cap = cv2.VideoCapture(0)


if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
   
    ret, frame = cap.read()

    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    
    faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        
        face_roi = gray_frame[y:y + h, x:x + w]

    
        img = resize(face_roi, (48, 48,3), anti_aliasing=True)

        
        img = color.rgb2gray(img)

  
        hog_features = hog(img, visualize=False)
        hog_features = hog_features.reshape(1, -1) / 225.0

        
        expression_prediction = modelq.predict(hog_features)

        
        emotion_label = label_dict[np.argmax(expression_prediction)]

       
        cv2.rectangle(frame, (x, y), (x + w, y + h), (250, 13, 12), thickness=3)

        
        txt = f"Expression: {emotion_label}"
        position = (x, y - 10)
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        txt_color = (0, 0, 0)
        outline_color = (255, 255, 0)
        cv2.putText(frame, txt, position, font, font_scale, outline_color, 5)
        cv2.putText(frame, txt, position, font, font_scale, txt_color, 2)

    
    cv2.imshow('Emotion Detector', frame)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 17ms/step
